In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint
import numpy as np
from pylab import savefig

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1) Import the WeatherPy_Database.csv file from your Weather_Database folder from Deliverable 1 as a DataFrame.
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,29.41,100,90,8.05,overcast clouds
1,1,Norman Wells,CA,65.2820,-126.8329,17.64,93,90,3.44,light snow
2,2,Ushuaia,AR,-54.8000,-68.3000,49.66,62,20,9.22,few clouds
3,3,Torbay,CA,47.6666,-52.7314,34.57,97,90,11.50,light rain
4,4,Viedma,AR,-40.8135,-62.9967,51.04,69,14,7.70,few clouds


In [3]:
# 2) write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 69
What is the maximum temperature you would like for your trip? 96


In [4]:
# 3) Use the loc method to filter the city_data_df DataFrame for temperature criteria collected in Step 2, then create a new DataFrame.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hilo,US,19.7297,-155.0900,82.27,77,1,3.44,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,75.07,81,64,10.78,light rain
10,10,Inirida,CO,3.8653,-67.9239,72.77,97,76,3.13,broken clouds
11,11,Vaini,TO,-21.2000,-175.2000,80.76,89,40,9.22,light rain
13,13,Georgetown,MY,5.4112,100.3354,89.55,70,40,1.99,scattered clouds
14,14,Puerto Ayora,EC,-0.7393,-90.3518,69.21,74,98,9.33,overcast clouds
15,15,Atuona,PF,-9.8000,-139.0333,78.33,74,5,16.73,clear sky
24,24,Kapaa,US,22.0752,-159.3190,80.58,81,90,3.00,light rain
25,25,Harper,LR,4.3750,-7.7169,77.63,88,69,7.16,light rain
29,29,Bubaque,GW,11.2833,-15.8333,83.46,76,7,9.60,clear sky


In [5]:
# 4a) Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [6]:
# 4b) Drop any empty rows (if found) and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                184
City                   184
Country                184
Lat                    184
Lng                    184
Max Temp               184
Humidity               184
Cloudiness             184
Wind Speed             184
Current Description    184
dtype: int64

In [7]:
# 5a) Use the provided code to create a new DataFrame called hotel_df to store values like city, country, coodinates and max temp.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b) Make a new column called "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,82.27,clear sky,19.7297,-155.0900,
9,Rikitea,PF,75.07,light rain,-23.1203,-134.9692,
10,Inirida,CO,72.77,broken clouds,3.8653,-67.9239,
11,Vaini,TO,80.76,light rain,-21.2000,-175.2000,
13,Georgetown,MY,89.55,scattered clouds,5.4112,100.3354,
14,Puerto Ayora,EC,69.21,overcast clouds,-0.7393,-90.3518,
15,Atuona,PF,78.33,clear sky,-9.8000,-139.0333,
24,Kapaa,US,80.58,light rain,22.0752,-159.3190,
25,Harper,LR,77.63,light rain,4.3750,-7.7169,
29,Bubaque,GW,83.46,clear sky,11.2833,-15.8333,


In [32]:
# 6a) Using the parameters search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#6b) Iterate through the hotel_df DataFrame.
for index, row in hotel_df.iterrows():
    
    #6c) Retrieve the latitude and longitude of each city
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Remember to add this new row to the Params!
    params["locations"]= f"{lat},{lng}"
    
#6d) Create the base URL Google API.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

#6e) Add the hotel name to the hotel_df DataFrame
hotels = requests.get(base_url, params=params).json()
results = hotels["results"]


#6f) If a hotel isn't found, skip to the next city.

try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.


In [33]:
hotels
#Google Direction API is enable but won't give output.

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [34]:
# 7) Remove the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.count()

City                   184
Country                184
Max Temp               184
Current Description    184
Lat                    184
Lng                    184
Hotel Name             183
dtype: int64

In [35]:
#Review that CSV file has proper format
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hilo,US,82.27,clear sky,19.7297,-155.0900,
9,Rikitea,PF,75.07,light rain,-23.1203,-134.9692,
10,Inirida,CO,72.77,broken clouds,3.8653,-67.9239,
11,Vaini,TO,80.76,light rain,-21.2000,-175.2000,
13,Georgetown,MY,89.55,scattered clouds,5.4112,100.3354,


In [36]:
# 8a. Create the output File (CSV)
clean_hotel_df=hotel_df.copy()
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [39]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

#Save figure ( a manual screen shot)
savefig("../Vacation_Search/WeatherPy_vacation_map.png")

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>